# L2. Docker

Источники:
 - [Полное практическое руководство по Docker: с нуля до кластера на AWS @ Хабр](https://habr.com/ru/articles/310460/)
 - [Основы контейнеризации (обзор Docker и Podman) @ Хабр](https://habr.com/ru/articles/659049/)
 - [Руководство по Docker Compose для начинающих @ Хабр](https://habr.com/ru/companies/ruvds/articles/450312/)

> [Docker](https://ru.wikipedia.org/wiki/Docker) — программное обеспечение для автоматизации развёртывания и управления приложениями в средах с поддержкой контейнеризации, контейнеризатор приложений. Позволяет «упаковать» приложение со всем его окружением и зависимостями в контейнер, который может быть развёрнут на любой Linux-системе с поддержкой контрольных групп в ядре, а также предоставляет набор команд для управления этими контейнерами. Изначально использовал возможности LXC, с 2015 года начал использовать собственную библиотеку, абстрагирующую виртуализационные возможности ядра Linux — libcontainer. С появлением Open Container Initiative начался переход от монолитной к модульной архитектуре. Разрабатывается и поддерживается одноимённой компанией-стартапом, распространяется в двух редакциях — общественной (Community Edition) по лицензии Apache 2.0 и для организаций (Enterprise Edition) по проприетарной лицензии. Написан на языке Go.

Докер это инструмент, который позволяет разработчикам, системными администраторам и другим специалистам деплоить их приложения в песочнице (которые называются контейнерами), для запуска на целевой операционной системе, например, Linux. Ключевое преимущество докера в том, что он позволяет пользователям упаковать приложение со всеми его зависимостями в стандартизированный модуль для разработки. В отличие от виртуальных машин, контейнеры не создают такой дополнительной нагрузки, поэтому с ними можно использовать систему и ресурсы более эффективно.

## История

Идея изоляции пользовательских пространств берет свое начало в 1979 году, когда в ядре UNIX появился системный вызов `chroot`. Он позволял изменить путь каталога корня `/` для группы процессов на новую локацию в файловой системе, то есть фактически создавал новый корневой каталог, который был изолирован от первого. Следующим шагом и логическим продолжением `chroot` стало создание в 2000 году FreeBSD jails («тюрем»), в которых изначально появилась частичная изоляция сетевых интерфейсов. В первой половине нулевых технологии виртуализации на уровне ОС активно развивались – появились Linux VServer (2001), Solaris Containers (2004) и OpenVZ (2005).

В операционной системе Linux технологии изоляции и виртуализации ресурсов вышли на новый этап в 2002 году, когда в ядро было добавлено первое пространство имен для изоляции файловой системы – `mount`. В 2006-2007 годах компанией Google был разработан механизм Process Containers (позднее переименованный в `cgroups`), который позволил ограничить и изолировать использование группой процессов ЦПУ, ОЗУ и др. аппаратных ресурсов. В 2008 году функционал `cgroups` был добавлен в ядро Linux. Достаточная функциональность для полной изоляции и безопасной работы контейнеров была завершена в 2013 году с добавлением в ядро пространства имен пользователей – `user`.

В 2008 году была представлена система LXC (LinuX Containers), которая позволила запускать несколько изолированных Linux систем (контейнеров) на одном сервере. LXC использовала для работы механизмы изоляции ядра – `namespaces` и `cgroups`. В 2013 году на свет появилась платформа `Docker`, невиданно популяризовавшая контейнерные технологии за счет простоты использования и широкого функционала. Изначально `Docker` использовал LXC для запуска контейнеров, однако позднее перешел на собственную библиотеку libcontainer, также завязанную на функционал ядра Linux. Наконец, в 2015 появился проект Open Container Initiative (OCI), который регламентирует и стандартизирует развитие контейнерных технологий по сей день.

Подробнее: [Недостающее введение в контейнеризацию @ Хабр](https://habr.com/ru/articles/541288/)

## Что такое контейнеры?  

Контейнеризация (виртуализация на уровне ОС) – технология, которая позволяет запускать программное обеспечение в изолированных на уровне операционной системы пространствах. Контейнеры являются наиболее распространенной формой виртуализации на уровне ОС. С помощью контейнеров можно запустить несколько приложений на одном сервере (хостовой машине), изолируя их друг от друга.

<img width="1000" alt="Сравнение докера с виртуальными машинами" src="./data/images/ait2_l2/docker_vs_vm.svg" style="margin:auto;">
<p style="text-align: center">
    Сравнение докера с виртуальными машинами
</p>

Процесс, запущенный в контейнере, выполняется внутри операционной системы хостовой машины, но при этом он изолирован от остальных процессов. Для самого процесса это вылядит так, будто он единственный работает в системе.

## Механизмы изоляции контейнеров

Изоляция процессов в контейнерах осуществляется благодаря двум механизмам ядра Linux – пространствам имен (`namespaces`) и контрольным группам (`cgroups`).
Пространства имен гарантируют, что процесс будет работать с собственным представлением системы. Существует несколько типов пространств имен:
 - файловая система (`mount`, `mnt`) – изолирует файловую систему
 - UTS (UNIX Time-Sharing, `uts`) – изолирует имя хоста и доменное имя
 - идентификатор процессов (process identifier, `pid`) – изолирует процессы
 - сеть (network, `net`) – изолирует сетевые интерфейсы
 - межпроцессное взаимодействие (`ipc`) – изолирует конкурирующее взаимодействие процессами
 - пользовательские идентификаторы (`user`) – изолирует ID пользователей и групп

Контрольные группы гарантируют, что процесс не будет конкурировать за ресурсы, зарезервированные за другими процессами. Они ограничивают (контролируют) объем ресурсов, который процесс может потреблять – ЦПУ, ОЗУ, пропускную способность сети и др.

Подробнее:
- [Механизмы контейнеризации: namespaces @ Хабр](https://habr.com/ru/company/selectel/blog/279281/)
- [Механизмы контейнеризации: cgroups @ Хабр](https://habr.com/ru/company/selectel/blog/303190/)

## Основные понятия

> **Container image** (образ) – файл, в который упаковано приложение и его среда. Он содержит файловую систему, которая будет доступна приложению, и другие метаданные (например команды, которые должны быть выполнены при запуске контейнера). Образы контейнеров состоят из слоев (как правило один слой – одна инструкция). Разные образы могут содержать одни и те же слои, поскольку каждый слой надстроен поверх другого образа, а два разных образа могут использовать один и тот же родительский образ в качестве основы. Образы хранятся в Registry Server (реестре) и версионируются с помощью tag (тегов). Если тег не указан, то по умолчанию используется latest. Примеры: [Ubuntu @ DockerHub](https://hub.docker.com/_/ubuntu), [Postgres @ DockerHub](https://hub.docker.com/_/postgres), [NGINX @ DockerHub](https://hub.docker.com/_/nginx).

> **Registry Server** (реестр, хранилище) – это репозиторий, в котором хранятся образы. После создания образа на локальном компьютере его можно отправить (`push`) в хранилище, а затем извлечь (`pull`) на другом компьютере и запустить его там. Существуют общедоступные и закрытые реестры образов. Примеры: [Docker Hub](https://hub.docker.com/) (репозитории docker.io), [RedHat Quay.io](https://quay.io/search) (репозитории quay.io).

> **Container** (контейнер) – это экземпляр образа контейнера. Выполняемый контейнер – это запущенный процесс, изолированный от других процессов на сервере и ограниченный выделенным объемом ресурсов (ЦПУ, ОЗУ, диска и др.). Выполняемый контейнер сохраняет все слои образа с доступом на чтение и формирует сверху свой исполняемый слой с доступом на запись.

> **Container Engine** (движок контейнеризации) – это программная платформа для упаковки, распространения и выполнения приложений, которая скачивает образы и с пользовательской точки зрения запускает контейнеры (на самом деле за создание и запуск контейнеров отвечает **Container Runtime**). Примеры: [Docker](https://docs.docker.com/get-started/overview/), [Podman](https://docs.podman.io/en/latest/).

> **Container Runtime** (среда выполнения контейнеров) – программный компонент для создания и запуска контейнеров. Примеры: [runc](https://github.com/opencontainers/runc) (инструмент командной строки, основанный на упоминавшейся выше библиотеке `libcontainer`), [crun](https://github.com/containers/crun).

> **Host** (хост) – сервер, на котором запущен **Container Engine** и выполняются контейнеры.

[Open Container Initiative](https://opencontainers.org/) (OCI) – это проект Linux Foundation, основанный в 2015 году компанией Docker, Inc, целью которого является разработка стандартов контейнеризации. В настоящее время в проекте участвуют такие компании, как Google, RedHat, Microsoft и др. OCI поддерживает спецификации [image-spec](https://github.com/opencontainers/image-spec) (формат образов) и [runtime-speс](https://github.com/opencontainers/runtime-spec) (Container Runtime).

<img width="1000" alt="Взаимодействие с Docker" src="./data/images/ait2_l2/docker_exchange_schema.svg" style="margin:auto;">
<p style="text-align: center">
    Взаимодействие с Docker
</p>


## Подсказки перед практикой

На практике при работе с контейнерами могут быть полезны следующие советы:
 - Простейший сценарий – скачать образ, создать контейнер и запустить его (выполнить команду внутри)
 - Документацию по запуску контейнера (путь к образу и необходимые команды с ключами) как правило можно найти в реестре образов (например, у Docker Hub есть очень удобный поисковик) или в ReadMe репозитория с исходным кодом проекта.
   > Создать образ и сохранить его в публичный реестр может практически каждый, поэтому старайтесь пользоваться только официальной документацией и проверенными образами!
 - Для скачивания образов используется команда `pull`, однако в целом она необязательна – при выполнении большинства команд (`create`, `run` и др.) образ скачается автоматически, если не будет обнаружен локально
 - При выполнении команд `pull`, `create`, `run` и др. следует указывать репозиторий и тег образа. Если этого не делать, то будут использоваться значения по умолчанию – репозиторий как правило `docker.io`, а тег `latest`
 - При запуске контейнера выполняется команда по умолчанию (точка входа), однако можно выполнить и другую команду

## Работа с Docker

Docker – это открытая платформа для разработки, доставки и запуска приложений. Состоит из утилиты командной строки `docker`, которая вызывает одноименный сервис (сервис является потенциальной единой точкой отказа) и требует права доступа `root`. По умолчанию использует в качестве **Container Runtime** `runc`. Все файлы Docker (образы, контейнеры и др.) по умолчанию хранятся в каталоге `/var/lib/docker`.

Для установки необходимо воспользоваться официальным руководством – [Download and install Docker](https://docs.docker.com/get-started/#download-and-install-docker), которое содержит подробные инструкции для Linux, Windows и Mac. Стоит сразу отметить, что контейнерам для работы необходимы функции ядра Linux, поэтому они работают нативно под Linux, почти нативно в последних версиях Windows благодаря WSL2 (через Docker Desktop или Linux диструбутив) и не нативно под Mac (используется виртуализация). Автор рекомендует использовать в тестовой и особенно в промышленной эксплуатации только Linux.

### Краткий гайд по установке Docker на Ubuntu

Удаляем старые версии:

```bash
sudo apt remove docker docker-engine docker.io containerd runc
```

Добавялем репозиторий:

```bash
sudo apt update
sudo apt install ca-certificates curl gnupg lsb-release
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
```

Устанавливаем `Docker Engine`:

```bash
sudo apt install docker-ce docker-ce-cli containerd.io docker-compose-plugin
```

Проверяем работоспособность с использованием образа `hello-world`:

```bash
sudo docker run hello-world
```

Запуск докера без запроса прав рута:
 - создаем группу пользователей `docker`
 - добавляем себя в эту группу
 - устанавливаем владельца файла `.docker` в папке юзера
 - устанавливаем права доступа файлу `.docker`

```bash
sudo groupadd docker
sudo usermod -aG docker $USER
sudo chown "$USER":"$USER" /home/"$USER"/.docker -R
sudo chmod g+rwx "$HOME/.docker" -R
```

Необходимо разлогиниться и залогиниться обратно. Проверка:

```bash
docker run hello-world
```

### Основные команды

```bash
# справочная информация
docker --help # список доступных команд
docker <command> --help # информация по команде
 
docker --version # версия Docker
docker info # общая информация о системе
 
# работа с образами
docker search debian # поиск образов по ключевому слову debian
 
docker pull ubuntu # скачивание последней версии (тег по умолчанию latest) официального образа ubuntu (издатель не указывается) из репозитория по умолчанию docker.io/library
docker pull prom/prometheus # скачивание последней версии (latest) образа prometheus от издателя prom из репозитория docker.io/prom
docker pull docker.io/library/ubuntu:18.04 # скачивание из репозитория docker.io официального образа ubuntu с тегом 18.04
 
docker images # просмотр локальных образов
 
docker rmi <image_name>:<tag> # удаление образа. Вместо <image_name>:<tag> можно указать <image_id>. Для удаления образа все контейнеры на его основе должны быть как минимум остановлены
docker rmi $(docker images -aq) # удаление всех образов
 
# работа с контейнерами
docker run hello-world # Hello, world! в мире контейнеров
docker run -it ubuntu bash # запуск контейнера ubuntu и выполнение команды bash в интерактивном режиме
docker run --name docker-getting-started --publish 8080:80 docker/getting-started # запуск контейнера gettind-started с отображением (маппингом) порта 8080 хоста на порт 80 внутрь контейнера
docker run --detach --name mongodb docker.io/library/mongo:4.4.10 # запуск контейнера mongodb с именем mongodb в фоновом режиме. Данные будут удалены при удалении контейнера!
 
docker ps # просмотр запущенных контейнеров
docker ps -a # просмотр всех контейнеров (в том числе остановленных)
docker stats --no-stream # просмотр статистики
 
docker start alpine # создание контейнера из образа alpine
 
docker start <container_name> # запуск созданного контейнера. Вместо <container_name> можно указать <container_id>
docker start $(docker ps -a -q) # запуск всех созданных контейнеров
 
docker stop <container_name> # остановка контейнера. Вместо <container_name> можно указать <container_id>
docker stop $(docker ps -a -q) # остановка всех контейнеров
 
docker rm <container_name> # удаление контейнера. Вместо <container_name> можно указать <container_id>
docker rm $(docker ps -a -q) # удаление всех контейнеров
 
# система
docker system info # общая информация о системе (соответствует docker info)
docker system df # занятое место на диске
docker system prune -af # удаление неиспользуемых данных и очистка диска
```

### Хранение данных

При запуске контейнер получает доступ на чтение ко всем слоям образа, а также создает свой исполняемый слой с возможностью создавать, обновлять и удалять файлы. Все эти изменения не будут видны для файловой системы хоста и других контейнеров, даже если они используют тот же базовый образ. При удалении контейнера все измененные данные также будут удалены. В большинстве случаев это предпочтительное поведение, однако иногда данные необходимо расшарить между несколькими контейнерами или просто сохранить.

Рассмотрим два способа хранения данных контейнеров:
 - [named volumes](https://docs.docker.com/get-started/05_persisting_data/) – именованные тома хранения данных  
   Позволяет сохранять данные в именованный том, который располагается в каталоге `/var/lib/docker/volumes` и не удаляется при удалении контейнера. Том может быть подключен к нескольким контейнерам
 - [bind mount](https://docs.docker.com/get-started/06_bind_mounts/) – монтирование каталога с хоста  
   Позволяет монтировать файл или каталог с хоста в контейнер. На практике используется для проброса конфигурационных файлов или каталога БД внутрь контейнера

Ниже приведены примеры использования `named volume` и `bind mount`:

```bash
# справочная информация
docker <command> --help
 
# named volume
docker run --detach --name jenkins --publish 80:8080 --volume=jenkins_home:/var/jenkins_home/ jenkins/jenkins:lts-jdk11 # запуск контейнера jenkins с подключением каталога /var/jenkins_home как тома jenkins_home
docker volume ls # просмотр томов
docker volume prune # удаление неиспользуемых томов и очистка диска. Для удаления тома все контейнеры, в которых он подключен, должны быть остановлены и удалены
 
# bind mount
# запуск контейнера node-exporter с монтированием каталогов внутрь контейнера в режиме read only: /proc хоста прокидывается в /host/proc:ro внутрь контейнера, /sys - в /host/sys:ro, а / - в /rootfs:ro
docker run \
-p 9100:9100 \
-v "/proc:/host/proc:ro" \
-v "/sys:/host/sys:ro" \
-v "/:/rootfs:ro" \
--name node-exporter prom/node-exporter:v1.1.2
```

Подробнее: [Хранение данных в Docker](https://habr.com/ru/company/southbridge/blog/534334/)

### Создание образа (Dockerfile)

Создание и распространение образов – одна из основных задач Docker. Рассмотрим два способа создания образа:
 - commit изменений из контейнера  
   Необходимо запустить контейнер из базового образа в интерактивном режиме, внести изменения и сохранить результат в образ с помощью команды `commit`. На практике способ удобен для небольших быстрых доработок.
 - декларативное описание через `Dockerfile`  
   Основной способ создания образов. Необходимо создать файл `Dockerfile` с декларативным описанием в формате `yaml` через текстовый редактор и запустить сборку образа командой `build`.

#### Пример c использованием `commit`

```bash
# справочная информация
docker <command> --help
 
# commit
# запуск контейнера из образа ubuntu в интерактивном режиме, установка утилиты ping и коммит образа под именем ubuntu-ping:20.04
docker run -it --name ubuntu-ping ubuntu:20.04 bash
apt update && apt install -y iputils-ping
exit
docker commit ubuntu-ping ubuntu-ping:20.04
docker images
```

#### Пример с использованием `Dockerfile`

Содержимое `Dockerfile`:

```dockerfile
# Dockerfile
# создание файла Dockerfile декларативного описания
FROM ubuntu:20.04
RUN apt-get update && apt-get install -y iputils-ping
```

Билд:

```bash
# запуск команды build из каталога с Dockerfile для создания образа simust/ubuntu-ping:20.04
docker build -t ubuntu-ping:20.04 .
docker images
```

Публикование образа:

```bash
# tag, login, push
docker tag ubuntu-ping:20.04 simust/ubuntu-ping:20.04 # создание из локального образа ubuntu-ping:20.04 тега с репозиторием для издателя simust
docker images
# вход в репозиторий docker.io под пользователем simust и отправка образа
docker login -u simust docker.io
docker push simust/ubuntu-ping:20.04
```

Подробнее: [Изучаем Docker, часть 3: файлы Dockerfile](https://habr.com/ru/company/ruvds/blog/439980/)

### Пример 1

Скрипт для билда контейнера:

```bash
#!/bin/sh
echo Setting env vars


export ubuntu_ver=18.04 # For docker image pulling
export cuda_ver=10.0
export image_tag=openmvg_18.04
export dockerfile=Dockerfile

# Check if image is existed before pulling
if docker image inspect ubuntu:$ubuntu_ver 1>/dev/null; then
  echo "Docker image ubuntu:$ubuntu_ver is found."
else
  echo "Pulling docker image ubuntu:$ubuntu_ver..."
  docker pull ubuntu:$ubuntu_ver
fi

echo Building docker
docker build --tag $image_tag \
             --build-arg ubuntu_ver=$ubuntu_ver \
			 --build-arg cuda_ver=$cuda_ver \
			 -f $dockerfile .
```

Докер-файл с инструкциями для сборки:

```dockerfile
ARG ubuntu_ver
FROM ubuntu:$ubuntu_ver
WORKDIR /
# Update and upgrade
RUN apt update && apt -y upgrade
ENV TZ=Europe/Samara \
    DEBIAN_FRONTEND=noninteractive

RUN apt install tzdata

# Create dir

RUN mkdir /usr/local/OpenMVG

# Python 3
RUN apt install -y curl python3-testresources python3-dev wget gnupg2 software-properties-common
WORKDIR /usr/local/OpenMVG/
RUN curl https://bootstrap.pypa.io/pip/3.6/get-pip.py -o get-pip.py && \
    python3 get-pip.py

RUN pip3 install Pillow

RUN apt install -y libpng-dev libjpeg-dev libtiff-dev libxxf86vm1 libxxf86vm-dev libxi-dev libxrandr-dev graphviz
RUN apt install -y libboost-all-dev libopencv-dev libatlas-base-dev

# CUDA installation

# First call to $ubuntu_ver is reset after FROM statement
ARG ubuntu_ver
ARG cuda_ver

RUN apt-key del 7fa2af80


RUN wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-keyring_1.0-1_all.deb && \
    dpkg -i cuda-keyring_1.0-1_all.deb
RUN apt update && apt -y install cuda=${cuda_ver}*

CMD [ "bash" ]
```

Скрипт для запуска контейнера:

```bash
docker run -v /home/ud/Documents/ROI_PM_FTMS/:/usr/local/OpenMVG/ -v /home/ud/Documents/boat/in/:/usr/local/in/ -v /home/ud/Documents/boat/out/:/usr/local/out -it openmvg_18.04 
```

### Пример 2

Докер-файл для сборки контейнера:

```dockerfile
FROM tensorflow/tensorflow:2.1.0-gpu-py3-jupyter
RUN add-apt-repository universe && apt install -y mc nano
RUN apt update && apt install -y libsm6 libxext6 libxrender-dev nano systemd redis-server net-tools libffi-dev libssl-dev python-openssl libjpeg-dev libpng-dev libtiff-dev
ADD redis.conf /etc/redis/redis.conf
#RUN pip3 install dramatiq[redis,watch]==1.8.1 imageio==2.5.0 numpy==1.13.3 opencv_contrib_python==4.1.0.25 scipy==1.4.1 six==1.11.0
RUN pip3 install --upgrade pip
RUN pip3 install --upgrade dramatiq[redis,watch]==1.8.1 imageio numpy opencv_contrib_python==4.1.0.25 scipy==1.4.1 six pandas scikit-image flask sshtunnel jsonpickle
WORKDIR usr/local/cnn
ENTRYPOINT /usr/bin/redis-server /etc/redis/redis.conf --daemonize yes && /bin/bash
CMD [ "bash" ]
```

Скрипт для запуска:

```bash
docker container run --name cubesat_doer_unet -v $(pwd)/cnn/:/usr/local/cnn/ --net cnn --ip 172.18.0.2 -p 6380:6380 --runtime=nvidia -it --rm cubesat_doer_unet /bin/bash
```

### Пример 3

Вспомогательный скрипт для установки занчений переменных среды в `RUN` во время сборки контейнера:

```bash
#!/bin/sh
# Set environment settings during RUN command execution for image building
# Used in cmake configuration, install and post-install commands

export py3_ver_mm_wod=$(python3 -c "import sys; print(\"\".join(map(str, sys.version_info[:2])))")
export py3_ver_mm=$(python3 -c "import sys; print(\".\".join(map(str, sys.version_info[:2])))")
export py3_ver_mmm=$(python3 -c "import sys; print(\".\".join(map(str, sys.version_info[:3])))")
export py3_inc_dir=$(python3 -c "from distutils.sysconfig import get_python_inc; print(get_python_inc())")
export py3_lib_dir=$(python3 -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())")
export py3_np_inc_dirs=${py3_lib_dir}/numpy/core/include

py3_lib_path=/usr/lib/x86_64-linux-gnu/libpython${py3_ver_mmm}.so

# TODO
if [ "${py3_ver_mm_wod}" = "36" ];
then
  py3_lib_path=$(locate "/usr/lib/x86_64-linux-gnu/libpython${py3_ver_mm}*.so");
fi

export py3_lib_path

printf "Environment vars for OpenCV build:\n  python3 versions: %s %s %s\n" "${py3_ver_mm_wod}" "${py3_ver_mm}" "${py3_ver_mmm}"
printf "  python3 include dir: %s\n" "${py3_inc_dir}"
printf "  python3 packages path dir: %s\n" "${py3_lib_dir}"
printf "  python3 numpy include dir: %s\n" "${py3_np_inc_dirs}"
printf "  python3 lib path: %s\n" "${py3_lib_path}"
```

Скрипт сборки:

```bash
#!/bin/sh
echo Setting env vars


export ubuntu_ver=18.04 # For docker image pulling
export cuda_ver=10.0
export qt_mm_ver=5.15
export qt_rev_ver=1
export ocv_ver=4.2.0 # OpenCV version for wget, cmake configuration, install and post-install commands
export build_thread_count=10 # for make command
export image_tag=tarsier_18.04
export dockerfile=TarsierDockerfile

# Check if image is existed before pulling
if docker image inspect ubuntu:$ubuntu_ver 1>/dev/null; then
  echo "Docker image ubuntu:$ubuntu_ver is found."
else
  echo "Pulling docker image ubuntu:$ubuntu_ver..."
  docker pull ubuntu:$ubuntu_ver
fi

echo Building docker
docker build --tag $image_tag \
             --build-arg ubuntu_ver=$ubuntu_ver \
			 --build-arg cuda_ver=$cuda_ver \
			 --build-arg qt_mm_ver=$qt_mm_ver \
			 --build-arg qt_rev_ver=$qt_rev_ver \
			 --build-arg ocv_ver=$ocv_ver \
			 --build-arg build_thread_count=$build_thread_count \
			 -f $dockerfile .
```

Содержимое докер-файла:

```dockerfile
ARG ubuntu_ver
FROM ubuntu:$ubuntu_ver
WORKDIR /
# Update and upgrade
RUN apt update && apt -y upgrade

# Create dir

RUN mkdir /usr/local/Dev

# Python 3
RUN apt install -y curl python3-testresources python3-dev wget gnupg2 software-properties-common
WORKDIR /usr/local/Dev/
RUN curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py && \
    python3 get-pip.py

# CUDA installation

# First call to $ubuntu_ver is reset after FROM statement
ARG ubuntu_ver
ARG cuda_ver

RUN if [ "$ubuntu_ver" = "18.04" ]; then \
      wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin && \
      mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600 && \
      apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub && \
      add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"; \
    elif [ "$ubuntu_ver" = "20.04" ]; then \
      wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin && \
      mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600 && \
      apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/7fa2af80.pub && \
      add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/ /"; \
    fi && \
    apt update && \
    apt -y install cuda=${cuda_ver}*

# OpenCV x.x.x with non free modules

RUN echo ttf-mscorefonts-installer msttcorefonts/accepted-mscorefonts-eula select true | debconf-set-selections

# To prevent interactive configuration of tzdata
ENV TZ=Europe/Samara
RUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone

## GStreamer

RUN apt -y install libgstreamer1.0-0 gstreamer1.0-plugins-base gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav gstreamer1.0-doc gstreamer1.0-tools gstreamer1.0-x gstreamer1.0-alsa gstreamer1.0-gl gstreamer1.0-gtk3 gstreamer1.0-qt5 gstreamer1.0-pulseaudio && \
    apt -y install ubuntu-restricted-extras libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libgstreamer-plugins-bad1.0-dev libgstreamer-plugins-bad1.0-0 libgstreamer-plugins-base1.0-0 libgstreamer-plugins-base1.0-dev

## OpenCV build dependencies

RUN apt -y install build-essential cmake unzip git pkg-config libgtk2.0-dev libavcodec-dev libavformat-dev \
                   libswscale-dev libtbb2 libtbb-dev libjpeg-dev libpng-dev libtiff-dev libdc1394-22-dev libv4l-dev \
                   libxvidcore-dev libx264-dev libgtk-3-dev libatlas-base-dev gfortran python-dev python-numpy \
                   python3-dev python3-pip python3-numpy

## OpenCV

ARG ocv_ver

RUN wget -O opencv.zip https://github.com/opencv/opencv/archive/${ocv_ver}.zip
RUN wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/${ocv_ver}.zip
RUN unzip opencv.zip
RUN unzip opencv_contrib.zip

RUN apt -y install mlocate && updatedb

COPY build_env.sh /usr/local/Dev
RUN chmod +x /usr/local/Dev/build_env.sh
RUN cd /usr/local/Dev/ && ./build_env.sh

WORKDIR /usr/local/Dev/opencv-${ocv_ver}
RUN mkdir build
WORKDIR /usr/local/Dev/opencv-${ocv_ver}/build

### Update numpy

RUN pip3 install -U numpy

ARG build_thread_count
ARG cmake_command

RUN apt-cache show cuda

RUN . /usr/local/Dev/build_env.sh && cmake_command="-D CMAKE_BUILD_TYPE=RELEASE \
-D CMAKE_INSTALL_PREFIX=/usr/local/OpenCV-${ocv_ver} \
-D OPENCV_SKIP_PYTHON_LOADER=OFF \
-D OPENCV_PYTHON3_INSTALL_PATH=/usr/local/OpenCV-${ocv_ver}/lib/python${py3_ver_mm}/site-packages \
-D OPENCV_PYTHON3_VERSION=${py3_ver_mmm} \
-D BUILD_opencv_python2=OFF \
-D BUILD_opencv_python3=ON \
-D BUILD_opencv_python_bindings_generator=ON \
-D PYTHON_DEFAULT_EXECUTABLE=$(which python3) \
-D PYTHON3_EXECUTABLE=$(which python3) \
-D PYTHON3_INCLUDE_DIR=${py3_inc_dir} \
-D PYTHON3_PACKAGES_PATH=${py3_lib_dir} \
-D PYTHON3_LIBRARY=${py3_lib_path} \
-D PYTHON3_NUMPY_INCLUDE_DIRS=${py3_np_inc_dirs} \
-D WITH_OPENCL=ON \
-D WITH_OPENMP=ON \
-D WITH_CUDA=ON \
-D WITH_CUDNN=OFF \
-D WITH_NVCUVID=OFF \
-D WITH_CUBLAS=ON \
-D WITH_GSTREAMER=ON \
-D ENABLE_FAST_MATH=1 \
-D CUDA_FAST_MATH=1 \
-D BUILD_opencv_cudacodec=OFF \
-D INSTALL_PYTHON_EXAMPLES=ON \
-D INSTALL_C_EXAMPLES=ON \
-D OPENCV_ENABLE_NONFREE=ON \
-D OPENCV_EXTRA_MODULES_PATH=/usr/local/Dev/opencv_contrib-${ocv_ver}/modules \
-D BUILD_EXAMPLES=ON .." && echo ${cmake_command} && cmake ${cmake_command}

RUN make -j${build_thread_count}
RUN make install
RUN ldconfig

RUN . /usr/local/Dev/build_env.sh && ln -sf /usr/local/OpenCV-${ocv_ver}/lib/python${py3_ver_mm}/site-packages/cv2/python-${py3_ver_mm}/$(ls /usr/local/OpenCV-${ocv_ver}/lib/python${py3_ver_mm}/site-packages/cv2/python-${py3_ver_mm}/) /usr/local/lib/python${py3_ver_mm}/dist-packages/cv2.so

RUN echo $(python3 -c "import cv2 as cv; print(cv.__version__)")

# Qt installation

ARG qt_mm_ver
ARG qt_rev_ver

WORKDIR /usr/local/Dev/
RUN wget https://download.qt.io/archive/qt/${qt_mm_ver}/${qt_mm_ver}.${qt_rev_ver}/single/qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}.tar.xz
RUN tar xf qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}.tar.xz

RUN apt -y install libgl1-mesa-dev libfontconfig1-dev libfreetype6-dev libx11-dev libx11-xcb-dev libxext-dev \
                   libxfixes-dev libxi-dev libxrender-dev libxcb1-dev libxcb-glx0-dev libxcb-keysyms1-dev \
				   libxcb-image0-dev libxcb-shm0-dev libxcb-icccm4-dev libxcb-sync0-dev libxcb-xfixes0-dev \
				   libxcb-shape0-dev libxcb-randr0-dev libxcb-render-util0-dev libxcb-xinerama0* \
				   libxkbcommon-dev libxkbcommon-x11-dev libatspi2.0-dev flite1-dev libspeechd-dev \
				   speech-dispatcher fontconfig libclang-dev gperf bison flex nodejs nodejs-dev libssl1.0-dev  \
				   llvm \
				   libasound2-dev libatkmm-1.6-dev libbz2-dev libcap-dev libcups2-dev libdrm-dev \
				   libfontconfig1-dev libfreetype6-dev libgcrypt11-dev libglu1-mesa-dev libicu-dev libnss3-dev \
				   libpci-dev libpulse-dev libudev-dev libx11-dev libxcb-composite0 \
				   libxcb-composite0-dev libxcb-cursor-dev libxcb-cursor0 libxcb-damage0 libxcb-damage0-dev \
				   libxcb-dpms0 libxcb-dpms0-dev libxcb-dri2-0 libxcb-dri2-0-dev libxcb-dri3-0 libxcb-dri3-dev \
				   libxcb-ewmh-dev libxcb-ewmh2 libxcb-glx0 libxcb-glx0-dev libxcb-icccm4 libxcb-icccm4-dev \
				   libxcb-image0 libxcb-image0-dev libxcb-keysyms1 libxcb-keysyms1-dev libxcb-present-dev \
				   libxcb-present0 libxcb-randr0 libxcb-randr0-dev libxcb-record0 libxcb-record0-dev \
				   libxcb-render-util0 libxcb-render-util0-dev libxcb-render0 libxcb-render0-dev libxcb-res0 \
				   libxcb-res0-dev libxcb-screensaver0 libxcb-screensaver0-dev libxcb-shape0 libxcb-shape0-dev \
				   libxcb-shm0 libxcb-shm0-dev libxcb-sync-dev libxcb-sync0-dev libxcb-sync1 libxcb-util-dev \
				   libxcb-util0-dev libxcb-util1 libxcb-xf86dri0 libxcb-xf86dri0-dev libxcb-xfixes0 \
				   libxcb-xfixes0-dev libxcb-xinerama0 libxcb-xinerama0-dev libxcb-xkb-dev libxcb-xkb1 \
				   libxcb-xtest0 libxcb-xtest0-dev libxcb-xv0 libxcb-xv0-dev libxcb-xvmc0 libxcb-xvmc0-dev \
				   libxcb1 libxcb1-dev libxcomposite-dev libxcursor-dev libxdamage-dev libxext-dev \
				   libxfixes-dev libxi-dev libxrandr-dev libxrender-dev libxslt-dev libxss-dev libxtst-dev

WORKDIR /usr/local/Dev/qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}
RUN mkdir build
WORKDIR /usr/local/Dev/qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}/build

RUN cd /usr/local/Dev/qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}/build && /usr/local/Dev/qt-everywhere-src-${qt_mm_ver}.${qt_rev_ver}/configure -prefix /usr/local/Qt${qt_mm_ver}.${qt_rev_ver} -platform linux-g++ -opensource -confirm-license
RUN make -j${build_thread_count}

#RUN pip3 install -U pyqt5 scipy colour-science scikit-image loguru pandas fast-slic
#RUN pip3 install imageio matplotlib numba oct2py pandas Pillow PyQt5 PyYAML

CMD [ "bash" ]
```

## Мультиконтейнерные приложения (Docker Compose)

[Docker Compose](https://docs.docker.com/compose/) – это инструмент для декларативного описания и запуска приложений, состоящих из нескольких контейнеров. Он использует `yaml` файл для настройки сервисов приложения и выполняет процесс создания и запуска всех контейнеров с помощью одной команды. Утилита `docker-compose` позволяет выполнять команды на нескольких контейнерах одновременно – создавать образы, масштабировать контейнеры, запускать остановленные контейнеры и др.

Одиночные контейнеры хорошо подходят для развертывания простейших приложений, работающих автономно, не зависящих, например, от внешних источников данных или от неких сервисов. На практике же подобные приложения - редкость. Реальные проекты обычно включают в себя целый набор совместно работающих приложений.  

Как узнать, нужно ли вам, при развёртывании некоего проекта, воспользоваться **Docker Compose**? На самом деле - очень просто. Если для обеспечения функционирования этого проекта используется несколько сервисов, то **Docker Compose** может вам пригодиться. Например, в ситуации, когда создают веб-сайт, которому, для выполнения аутентификации пользователей, нужно подключиться к базе данных. Подобный проект может состоять из двух сервисов - того, что обеспечивает работу сайта, и того, который отвечает за поддержку базы данных.

Технология **Docker Compose**, если описывать её упрощённо, позволяет, с помощью одной команды, запускать множество сервисов.

### Разница между Docker и Docker Compose

Docker применяется для управления отдельными контейнерами (сервисами), из которых состоит приложение.

Docker Compose используется для одновременного управления несколькими контейнерами, входящими в состав приложения. Этот инструмент предлагает те же возможности, что и Docker, но позволяет работать с более сложными приложениями.

<img width="1000" alt="Docker vs Docker Compose" src="./data/images/ait2_l2/docker_vs_docker_compose.svg" style="margin:auto;">
<p style="text-align: center">
    Docker vs Docker Compose
</p>

### Пример

Конфигурационный файл:

```yaml
version: "3"
# Следует учитывать, что docker-composes работает с сервисами.
# 1 сервис = 1 контейнер.
# Сервисом может быть клиент, сервер, сервер баз данных
# Раздел, в котором будут описаны сервисы, начинается с 'services'.
services:
  minio_server:
    container_name: hsis_minio
    image: quay.io/minio/minio
    environment:
      MINIO_ROOT_USER: $MINIO_ADMIN_USER
      MINIO_ROOT_PASSWORD: $MINIO_ADMIN_PASSWORD
      MINIO_DEFAULT_BUCKETS: $MINIO_BUCKET_NAME
    volumes:
      - $MINIO_REMOTE_VOL:$MINIO_LOCAL_VOL
    # Команда, которую нужно запустить после создания образа.
    command: server $MINIO_LOCAL_VOL --console-address :$MINIO_SRV_CONSOLE_LOCAL_PORT
    networks:
      hsis_net:
        ipv4_address: $DOCKER_MINIO_IP
    # Если мы хотим обратиться к серверу с нашего компьютера (находясь за пределами контейнера),
    # мы должны организовать перенаправление этого порта на порт компьютера.
    # Сделать это нам поможет ключевое слово 'ports'.
    # При его использовании применяется следующая конструкция: [порт компьютера]:[порт контейнера]
    ports:
      - $MINIO_SRV_CONSOLE_REMOTE_PORT:$MINIO_SRV_CONSOLE_LOCAL_PORT
      - $MINIO_SRV_API_REMOTE_PORT:$MINIO_SRV_API_LOCAL_PORT
  db:
    container_name: hsis_db
    image: postgres:bullseye
    environment:
      PGDATA: $PG_DATA
      POSTGRES_PASSWORD: $PG_PASSWORD
      POSTGRES_DB: $PG_DB_NAME
    volumes:
      - $PG_REMOTE_VOL:$PG_LOCAL_VOL
    networks:
      hsis_net:
        ipv4_address: $DOCKER_DB_IP
    ports:
      - $PG_REMOTE_PORT:$PG_LOCAL_PORT
    depends_on:
      - minio_server
  pgadmin:
    container_name: hsis_db_web
    image: dpage/pgadmin4:7.7
    environment:
      PGADMIN_DEFAULT_EMAIL: $PGA_EMAIL
      PGADMIN_DEFAULT_PASSWORD: $PGA_PASSWORD
    volumes:
      - $PGA_REMOTE_VOL:$PGA_LOCAL_VOL
    networks:
      hsis_net:
        ipv4_address: $DOCKER_DB_WEB_IP
    ports:
      - $PGA_WEB_REMOTE_PORT:$PGA_WEB_LOCAL_PORT
    # Ключевое слово 'depends_on' позволяет указывать, должен ли сервис,
    # прежде чем запуститься, ждать, когда будут готовы к работе другие сервисы.
    depends_on:
      - minio_server
      - db
  web:
    container_name: hsis_web
    # Ключевое слово "build" позволяет задать
    # путь к файлу Dockerfile, который нужно использовать для создания образа,
    # который позволит запустить сервис.
    # Здесь 'server/' соответствует пути к папке сервера,
    # которая содержит соответствующий Dockerfile.
    build:
      context: ..
      dockerfile: docker/hsi_storage_web.dockerfile
    environment:
      MODULE_NAME: hsis_server.__init__
      VARIABLE_NAME: app
#      LOG_LEVEL: "debug"
#      WORKERS_PER_CORE: 0.5
    networks:
      hsis_net:
        ipv4_address: $DOCKER_WEB_IP
    ports:
      - $DB_WEB_REMOTE_PORT:$DB_WEB_LOCAL_PORT
    depends_on:
      - minio_server
      - db
networks:
  hsis_net:
    ipam:
      driver: default
      config:
        - subnet: $DOCKER_NETWORK
```

Содержимое `docker/hsi_storage_web.dockerfile`:

```dockerfile
#FROM tiangolo/meinheld-gunicorn-flask:python3.9
FROM tecktron/python-waitress:latest
ENV APP_DIR /hsis_server
WORKDIR /
# Update and upgrade
RUN apt update && apt -y upgrade
RUN apt install -y wget
RUN mkdir -p $APP_DIR
COPY hsis_server/requirements.txt $APP_DIR
RUN pip install --no-cache-dir --upgrade -r /hsis_server/requirements.txt
COPY docker/hsis.env $APP_DIR
COPY docker/hsis_secrets.env $APP_DIR
COPY hsis_server $APP_DIR
```

Скрипт сборки:

```bash
#!/bin/sh
cp hsis.env .env
echo >> .env
cat hsis_secrets.env >> .env
echo Create dirs
export $(grep -v '^#' hsis.env | xargs)
mkdir -p -v $MINIO_REMOTE_VOL
mkdir -p -v $PGA_REMOTE_VOL
sudo chown -R 5050:5050 $PGA_REMOTE_VOL
echo Creating docker network
#docker network create --subnet=$DOCKER_MINIO_IP $DOCKER_NETWORK_NAME
echo Building docker
docker compose -f hsi_storage_compose.yml build
```

Запуск:

```bash
docker compose -f hsi_storage_compose.yml up
```